In [ ]:
import pandas as pd

# mitre attack

In [ ]:
import json
d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
d.keys()

## Изучение

In [ ]:
d['objects'][0].keys()

### типы объектов

In [ ]:
set([it['type'] for it in d['objects']])

### тактика

In [ ]:
tactic_l = [it for it in d['objects'] if it['type']=='x-mitre-tactic']


In [ ]:
[it for it in tactic_l if 'eva' in it['name']]

### пример

In [ ]:
it = d['objects'][1000]
it['type'], it['name'], it['description'], it['x_mitre_attack_spec_version'], it['x_mitre_version']

In [ ]:
it['object_marking_refs']

### поиск атаки

In [ ]:
text = open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json').read()

In [ ]:
len(text)

In [ ]:
ch = text.find('Adversaries may execute active reconnaissance scans to gather information that can be used during targeting.')

In [ ]:
text[ch-1200:ch+100]

## возьмем атаки

In [ ]:
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
len(attack_l)

In [ ]:
attack_l[0]


In [ ]:

# 1406, 1575
[it for it in attack_l if it['external_references'][0]['external_id']=='T1557.001']

In [ ]:
from itertools import chain
set(chain(*[it['x_mitre_domains'] for it in attack_l]))

In [ ]:
enterp_attack_l = [it for it in attack_l if 'enterprise-attack' in it['x_mitre_domains']]

enterp_attack_l это все, которые попали в наш фильтр, может тогда фильтр слишком жесткий?

In [ ]:
len(enterp_attack_l)

In [ ]:
# https://attack.mitre.org/techniques/T1059/
s = 'Adversaries may scan victim IP blocks to gather information that can be used during targeting.'
attack = [it for it in attack_l if s in it['description']]
attack

In [ ]:
attack[0]['description'], attack[0]['external_references'][0]['external_id'], attack[0]['external_references'][0]['url']

In [ ]:
mitre_df = pd.DataFrame([(attack['description'], attack['external_references'][0]['external_id'], attack['external_references'][0]['url']) for attack in attack_l],
                        columns=['sentence', 'labels', 'url'])

In [ ]:
s = 'Adversaries may scan victim IP blocks to gather information that can be used during targeting.'
mitre_df.query('sentence.str.contains(@s)')

In [ ]:
url = 'https://attack.mitre.org/tactics/TA0006/'
mitre_df.query('url==@url')

In [ ]:
mitre_df['labels']

## ищем по тексту

In [ ]:
text = open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json').read()

In [ ]:
len(text)

In [ ]:
s = 'has used watering hole attacks, often with zero-day exploits, to gain initial access'
ch = text.find(s)

In [ ]:
text[ch-150:ch+300]

In [ ]:
'https://attack.mitre.org/techniques/T1059/002/'

## ищем признак группы

In [ ]:
descr1 = 'may circumvent mechanisms designed to control elevate privileges to gain higher-level permissions.'
descr2 = 'Within Linux and MacOS systems, sudo'
descr3 = 'Adversaries may gain access and continuously communicate with victims by injecting malicious content into systems through online network traffic'

attack1 = [it for it in attack_l if descr1 in it['description']][0]
attack2 = [it for it in attack_l if descr2 in it['description']][0]
attack3 = [it for it in attack_l if descr3 in it['description']][0]

In [ ]:
attack2
# identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5

In [ ]:
kill_chain_phases


## Удалим ссылки из описания и соберем в отдельном поле

In [ ]:
mitre_df['descr_links'] = mitre_df['sentence'].str.findall('\((https://[^\s]{1,})\)')

In [ ]:
mitre_df['sentence'] = mitre_df['sentence'].str.replace(r'\(https://[^\s]{1,}\)', r'', regex=True)
mitre_df

### ссылки из процедур и митигаций

In [ ]:
attack_sample = mitre_df.query('labels=="T1548.003" and not proc_flag')
display(attack_sample)
id = attack_sample['id'].iloc[0]

In [ ]:
attack = [it for it in attack_l if it['id']==id][0]
attack

In [ ]:
attack['x_mitre_platforms'], attack['external_references'][1:], attack['kill_chain_phases'], attack['name'],
attack['x_mitre_data_sources'], attack['x_mitre_permissions_required'], attack['x_mitre_effective_permissions'], attack['created']

In [ ]:
[Proton](https://attack.mitre.org/software/S0279) modifies the tty_tickets line in the sudoers file.(Citation: objsee mac malware 2017)

In [ ]:
proc_software_links = []
proc_citation_links = []

proc_itms = [it for it in rel_l if (it['target_ref']==attack['id']) and (it['relationship_type']=='uses')]
it = proc_itms[1]
it

In [ ]:
import re
# proc_software_links, proc_descr_links
re.findall(r'\((https://[^\s\)]{1,})', it['description']), [subit['url'] for subit in it['external_references']]

In [ ]:
det_itms = [it for it in rel_l if (it['target_ref']==attack['id']) and (it['relationship_type']=='detects')]
it = det_itms[2]
# det_descr, det_data_comp
it['description'], it['source_ref']

In [ ]:
mit_itms = [it for it in rel_l if (it['target_ref']==attack['id']) and (it['relationship_type']=='mitigates')]
it = mit_itms[2]
it['description'], it['source_ref']

## ищем связанные процедуры с objectpath

In [ ]:
!pip install objectpath

In [ ]:
import objectpath as obp

query = obp.Tree(d)

### объект по описанию можно найти

In [ ]:
s = '[Andariel](https://attack.mitre.org/groups/G0138) has used watering hole attacks, often with zero-day exploits, to gain initial access to victims within a specific IP range.(Citation: AhnLab Andariel Subgroup of Lazarus June 2018)(Citation: TrendMicro New Andariel Tactics July 2018)'

res_descr = list(query.execute(f"$..*[@.description is '{s}']"))

In [ ]:
type(res_descr)

Связи с атакой:
- created_by_ref
- target_ref
- object_marking_refs

In [ ]:
attack = [it for it in attack_l if 'Typical drive-by compromise process:' in it['description']][0]
attack

#### created_by_ref
много очень

In [ ]:
ident = 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5'

res2 = list(query.execute(f"$..*[@.created_by_ref is '{ident}']"))


In [ ]:
res2[0].keys()

#### object_marking_refs
тут тоже дичь

In [ ]:
object_marking_refs = 'marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'

res2 = list(query.execute(f"$..*['{object_marking_refs}' in @.object_marking_refs]"))
res2

In [ ]:
len(res2[0])

In [ ]:
res2[0]['object_marking_refs']

### по id атаки

In [ ]:
res = list(query.execute("$..*[@.target_ref is 'attack-pattern--d742a578-d70e-4d0e-96a6-02a9c30204e6']"))

In [ ]:
{it['relationship_type'] for it in res}

Тут как раз все виды
- uses - процедуры
- detects - Detection
- mitigates - Mitigations

In [ ]:
(len([it for it in res if it['relationship_type']=='uses']), len([it for it in res if it['relationship_type']=='detects']),
len([it for it in res if it['relationship_type']=='mitigates']))

In [ ]:
[it for it in res if it['relationship_type']=='uses'][-1]

- некоторые примеры процедур почему-то на сайте не отображаются
  - [пример](https://attack.mitre.org/techniques/T1189/), а в ссылках есть (it['relationship_type']=='uses' с id атаки) и [тут](https://service.securitm.ru/mitre/techniques/T1189)

### поиск процедур
сначала нашел родителя, как в файле objectpath и обнаружил, что у него столько же элементов, сколько и у верхнеуровневого objects ключа и сделал вывод, что один

In [ ]:
obj_target_refs = [it for it in d['objects'] if 'target_ref' in it.keys()]

In [ ]:
pattern = 'attack-pattern--d742a578-d70e-4d0e-96a6-02a9c30204e6'


In [ ]:
rel_l = [it for it in d['objects'] if it['type']=='relationship']

По ходу это объекты с типом relationship

In [ ]:
len(obj_target_refs), len(rel_l)

количество совпало

In [ ]:
[it for it in rel_l if not 'target_ref' in it.keys()]

нет item-ов без ссылок атаку target_ref

In [ ]:
attack

## атаки одноименки

In [ ]:
import json
import re

d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
# attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern' if not 'revoked' in it.keys() or not it['revoked']]
rel_l = [it for it in d['objects'] if it['type']=='relationship']

mitre_df = pd.DataFrame([(attack['id'], attack['name'], attack['description'], attack['external_references'][0]['external_id'],
                          attack['external_references'][0]['url'], attack['x_mitre_platforms'], attack['kill_chain_phases'],
                          attack['created'], attack['external_references'][1:],
                          attack['x_mitre_permissions_required'] if 'x_mitre_permissions_required' in attack.keys() else None,
                         attack['x_mitre_effective_permissions'] if 'x_mitre_effective_permissions' in attack.keys() else None,
                          attack['x_mitre_data_sources'] if 'x_mitre_data_sources' in attack.keys() else None,
                          attack['x_mitre_defense_bypassed'] if 'x_mitre_defense_bypassed' in attack.keys() else None
                          )
                          for attack in attack_l],
                        columns=['id', 'name', 'sentence', 'labels', 'url', 'platforms', 'kill_chain_phases',
                                 'created', 'doc_refs', 'permissions', 'effective_permissions', 'data_sources', 'defense_bypassed'])
mitre_df['proc_flag'] = False


In [ ]:
mitre_df[mitre_df['name'].duplicated(keep=False)].sort_values(by='name').head()

## выделяем субтехники

In [ ]:
import json
import re

d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
# attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern' if not 'revoked' in it.keys() or not it['revoked']]

In [ ]:
len([it for it in attack_l if  it['x_mitre_is_subtechnique']==False])

## атаки с устаревшими ссылками

In [ ]:
d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']

### T1067

In [ ]:
prb_l = [it for it in attack_l if it['name']=='Bootkit']
len(prb_l)

In [ ]:
prb_l[0]

In [ ]:
prb_l[1]

### T1066

In [ ]:
prb_l = [it for it in attack_l if it['name']=='Indicator Removal from Tools']
prb_l[0]

In [ ]:
prb_l[1]

- отличие флаг 'revoked': False (актуальная)

In [ ]:
mitre_df.query('labels.isin(["T1027.005", "T1066"])')

### проблемные id атак

In [ ]:
d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
mitre_df = pd.DataFrame([(attack['id'], attack['description'], attack['external_references'][0]['external_id'], attack['external_references'][0]['url']) for attack in attack_l],
                        columns=['id', 'sentence', 'labels', 'url'])

In [ ]:
pattern = 'attack-pattern--00d0b012-8a03-410e-95de-5826bf542de6'

obj_target_refs = [it for it in d['objects'] if 'target_ref' in it.keys()]

In [ ]:
mitre_df.query(f'id=="{pattern}"')

In [ ]:
attack = [it for it in attack_l if it['id']==pattern][0]
attack

In [ ]:
res = list(query.execute(f"$..*[@.target_ref is '{pattern}']"))

In [ ]:
{it['relationship_type'] for it in res}

## Итог

In [ ]:
import json
import re

d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
# attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern' if not 'revoked' in it.keys() or not it['revoked']]
rel_l = [it for it in d['objects'] if it['type']=='relationship']

mitre_df = pd.DataFrame([(attack['id'], attack['name'], attack['description'], attack['external_references'][0]['external_id'],
                          attack['external_references'][0]['url'], attack['x_mitre_is_subtechnique'], attack['x_mitre_platforms'], attack['kill_chain_phases'],
                          attack['created'], attack['external_references'][1:],
                          attack['x_mitre_permissions_required'] if 'x_mitre_permissions_required' in attack.keys() else None,
                         attack['x_mitre_effective_permissions'] if 'x_mitre_effective_permissions' in attack.keys() else None,
                          attack['x_mitre_data_sources'] if 'x_mitre_data_sources' in attack.keys() else None,
                          attack['x_mitre_defense_bypassed'] if 'x_mitre_defense_bypassed' in attack.keys() else None
                          )
                          for attack in attack_l],
                        columns=['id', 'name', 'sentence', 'labels', 'url', 'subtechnique', 'platforms', 'kill_chain_phases',
                                 'created', 'doc_refs', 'permissions', 'effective_permissions', 'data_sources', 'defense_bypassed'])
mitre_df['proc_flag'] = False


In [ ]:
from itertools import chain
mittre_l = []
for _, row in mitre_df.iterrows():

    proc_itms = [it for it in rel_l if (it['target_ref']==row['id']) and (it['relationship_type']=='uses')]
    det_itms = [it for it in rel_l if (it['target_ref']==row['id']) and (it['relationship_type']=='detects')]
    mit_itms = [it for it in rel_l if (it['target_ref']==row['id']) and (it['relationship_type']=='mitigates')]


    part_df = row.drop(['proc_flag']).to_frame().T
    part_df['proc_software_links'] = [list(chain(*[re.findall(r'\((https://[^\s\)]{1,})', it['description']) for it in proc_itms ]))]

    part_df['proc_descr_links'] = [[subit['url'] for it in proc_itms if 'external_references' in it for subit in it['external_references'] ]]
    part_df['proc_descr'] = [[it['description'] for it in proc_itms]]
    # break

    part_df['det_descr'] = [[it['description'] for it in det_itms if 'description' in it]]
    part_df['det_data_comp'] = [[it['source_ref'] for it in det_itms if 'source_ref' in it]]
    part_df['mit_descr'] = [[it['description'] for it in mit_itms if 'description' in it]]
    part_df['mit_coa'] = [[it['source_ref'] for it in mit_itms if 'source_ref' in it]]

    mittre_l.append(part_df)




In [ ]:
mitre_df = pd.concat(mittre_l, ignore_index=True)


In [ ]:
mitre_df['kill_chain_tags'] = mitre_df['kill_chain_phases'].map(lambda x: [it['phase_name'] for it in x])

In [ ]:
mitre_df.head(2)

In [ ]:
import joblib
joblib.dump(mitre_df, '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_df.pkl')

In [ ]:
!grep -rl -E '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_df.pkl'  '/content/drive/MyDrive/Colab Notebooks'

Алан, добрый день.

Для получения доступа к ML Space вам необходимо обратиться к администратору организации, в которую вас пригласили, чтобы он выдал вам права на платформу:
https://cloud.ru/docs/administration/ug/topics/guides__employees_add.html

Создать новый проект может также администратор организации, вам необходимо обратиться к нему для создания нового проекта: https://cloud.ru/docs/administration/ug/topics/guides__projects.html#id1

### загрузка для предсказаний

In [ ]:
import pandas as pd

In [ ]:
import joblib
mitre_df = joblib.load('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_df.pkl')

In [ ]:
mitre_df

In [ ]:
proc_df = mitre_df[['id', 'name', 'url', 'labels', 'proc_descr', 'kill_chain_tags', 'subtechnique']].explode('proc_descr')\
                  .rename(columns={'proc_descr':'sentence'})\
                  .assign(is_proc=True)
main_descr_df = mitre_df[['id', 'name', 'url', 'labels', 'sentence', 'kill_chain_tags', 'subtechnique']].assign(is_proc=False)
mitre_attack_df = pd.concat([main_descr_df, proc_df], ignore_index=True)

In [ ]:
# удаляем строки, где процедуры не было совсем
mitre_attack_df = mitre_attack_df[~mitre_attack_df['sentence'].isna()].reset_index(drop=True)


In [ ]:
# удалим ссылочки
mitre_attack_df['sentence'] = mitre_attack_df['sentence'].str.replace(r'\(https://[^\s]{1,}\)', r'', regex=True)
mitre_attack_df.head(2)

In [ ]:
mitre_attack_df.query('labels=="T1574.002"').head(3)

In [ ]:
label2tactic = mitre_attack_df.set_index('labels')['kill_chain_tags'].to_dict()

#### dumps

In [ ]:
joblib.dump(mitre_attack_df[['sentence', 'labels']]\
        .assign(labels = mitre_attack_df['labels'].map(lambda x:[x])), '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_source.pkl')

In [ ]:
joblib.dump(label2tactic, '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/label2tactic.pkl')

### небольшая eda

In [ ]:
mitre_df.head(2)

#### data_sources

In [ ]:
mitre_df['data_sources'].isna().sum(), mitre_df['data_sources'].isna().mean()

In [ ]:
temp_df = mitre_df.explode('data_sources')
temp_df['data_sources'].nunique() , temp_df.shape[0], temp_df['data_sources'].nunique()/temp_df.shape[0]

In [ ]:
temp_df['data_sources'].duplicated().mean()

- Мало пропусков, 100 уникальных и почти каждая дублируется, может хорошо разделить данные

#### permissions

In [ ]:
mitre_df['effective_permissions'].isna().mean(), mitre_df['permissions'].isna().mean()

In [ ]:
temp_df = mitre_df.explode('permissions')
temp_df['permissions'].nunique() , temp_df.shape[0], temp_df['permissions'].nunique()/temp_df.shape[0]

- Мало записей с разрешениями
- уникальных мало, поэтому можно добавить к эмбеддингу

#### defense_bypassed

In [ ]:
mitre_df['defense_bypassed'].isna().sum(), mitre_df['defense_bypassed'].isna().mean()

In [ ]:
temp_df = mitre_df.explode('defense_bypassed')
temp_df['defense_bypassed'].nunique() , temp_df.shape[0], temp_df['defense_bypassed'].nunique()/temp_df.shape[0]

- Мало записей с обходом защиты
- уникальных 39, поэтому можно добавить к эмбеддингу

#### mit coa

In [ ]:
#	mit_coa
mitre_df.explode('mit_coa')

In [ ]:
mitre_df.explode('mit_coa')['mit_coa'].value_counts().loc[lambda x: x==1].sum(), mitre_df.explode('mit_coa').shape[0]

#### det_data_comp

In [ ]:
mitre_df.explode('det_data_comp')['det_data_comp'].value_counts().loc[lambda x: x==1].sum(), mitre_df.explode('det_data_comp').shape[0]

#### proc_descr_links

In [ ]:
mitre_df.explode('proc_descr_links')['proc_descr_links'].value_counts().loc[lambda x: x==1].sum(), mitre_df.explode('proc_descr_links').shape[0]

proc_descr_links содержит дубли если шо

#### proc_software_links

In [ ]:
# столько ссылок software встречаются однократно
mitre_df.explode('proc_software_links')['proc_software_links'].value_counts().loc[lambda x: x==1].sum()/mitre_df.explode('proc_software_links').shape[0]

In [ ]:
mitre_df.explode('proc_software_links')['proc_software_links'].nunique(), mitre_df.explode('proc_software_links').shape[0]

#### doc_refs

In [ ]:
# столько ссылок встречаются однократно
mitre_df.explode('doc_refs')['doc_refs'].value_counts().loc[lambda x: x==1].sum()/mitre_df.explode('doc_refs').shape[0], mitre_df.explode('doc_refs').shape[0]

In [ ]:
mitre_df['platforms'].explode().unique()

In [ ]:
mitre_df['kill_chain_tags'].explode().unique()

In [ ]:
# где пусто, там пустые списки
for col in ['proc_software_links', 'det_data_comp', 'mit_coa', 'proc_descr_links', 'doc_refs', 'platforms', 'kill_chain_tags']:

  display(mitre_df[mitre_df[col].map(lambda x: (len(x)==0) & (not isinstance(x, list)))])

- doc_refs немного, но повторения встречаютс только для половины
- proc_software_links в 10 раз больше, повторяются почти все
- proc_descr_links около 18 тыс. записей,
- det_data_comp, mit_coa поменьше можно

включаем в общий датасет пока proc_software_links, det_data_comp, mit_coa  и сцепляем. platforms и kill_chain_tags можно потом добавить one hot

#### subtechnique

In [ ]:
mitre_attack_df.loc[(mitre_attack_df.subtechnique!=True), 'labels'].str.len().describe()

In [ ]:
mitre_attack_df.loc[(mitre_attack_df.subtechnique==True), 'labels'].str.len().describe()

### после вытягивания процедур

In [ ]:
mitre_df['descr_links'] = mitre_df['sentence'].str.findall('\((https://[^\s]{1,})\)')

In [ ]:
mitre_df['sentence'] = mitre_df['sentence'].str.replace(r'\(https://[^\s]{1,}\)', r'', regex=True)
mitre_df

In [ ]:
with pd.option_context('display.max_colwidth', 400):
  display(mitre_df.query('sentence.str.contains("Binary padding effectively changes")'))

In [ ]:
with pd.option_context('display.max_colwidth', 400):
  display(mitre_df.query('id=="attack-pattern--5bfccc3f-2326-4112-86cc-c1ece9d8a2b5"'))

у кого нет процедур, возможно, ссылаются на другие атаки, у кого есть

## Проблемы

### одинаковое описание разные атаки

- https://attack.mitre.org/techniques/T1009
- https://attack.mitre.org/techniques/T1027.001

удалим почти дубли, надо подумать как

### на сайте чего-то нет, но есть в данных (правильно?)

- некоторые примеры процедур почему-то на сайте не отображаются
  - [пример](https://attack.mitre.org/techniques/T1189/), а в ссылках есть (it['relationship_type']=='uses' с id атаки) и [тут](https://service.securitm.ru/mitre/techniques/T1189)

### нет процедур

- хотя ссылаются на другие атаки
  - https://attack.mitre.org/techniques/T1066
  T1067

In [ ]:
mitre_df.query('labels.isin(["T1027.005", "T1066"])')[['name', 'id', 'url', 'mit_coa', 'kill_chain_tags']]

Одна запись ссылается на другую при этом у них разные mit_coa

- как оценить похожесть текстов

## найдем похожие атаки

- по разделу
- по общим сссылкам
- по общим описаниям

# tram

In [ ]:
tram_data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/scibert/multi_label.json').drop(columns='doc_title')
tram_data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/data_df.csv')
data['labels'] = data['labels'].map(lambda x: eval(x))
data

In [ ]:
data[data['labels'].map(lambda x: 'T1027.005' in x)]

In [ ]:
mitr_data[mitr_data['labels'].map(lambda x: 'T1027.005' in x)]

# sec bert


In [ ]:
mitr_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/dataset.txt')
mitr_data['labels'] = mitr_data[['label_subtec', 'label_tec']].apply(lambda x: list(set((x[0], x[1]))), axis=1)
mitr_data = mitr_data[['sentence', 'labels']]

In [ ]:
joblib.dump(mitr_data[['sentence', 'labels']], '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_alt.pkl')

# итоговая выгрузка на ноут

In [ ]:
import pandas as pd
import joblib
import numpy as np

In [ ]:
use_old_mitre = False

In [ ]:
tram_df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/scibert/multi_label.json').drop(columns='doc_title')


In [ ]:
if use_old_mitre:
  mitr_df = joblib.load('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_alt.pkl')
else:
  mitr_df = joblib.load('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_source.pkl')

In [ ]:
rep_data  = joblib.load('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/rep_data.pkl')
rep_data.head(2)

In [ ]:
df = pd.concat([mitr_df, tram_df], ignore_index=True)

In [ ]:
https://attack.mitre.org/techniques/T1574/002/

In [ ]:
with pd.option_context('display.max_colwidth', 500):
  display(tram_df.head())

In [ ]:
df = pd.concat([df, rep_data[['sentence', 'labels']]], ignore_index=True)

In [ ]:
target = 'tactic'
if target == 'tactic':
  from itertools import chain
  label2tactic = joblib.load('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/label2tactic.pkl')

  df['labels'] = df['labels'].map(lambda x: list(chain(*[label2tactic[it] for it in x])))


joblib.dump(df, '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_full_df.pkl')

In [ ]:
df

In [ ]:
df[df.sentence.str.contains('The downloader obtains and launches a vulnerable application')]


In [ ]:
df[df['sentence']=='Obfuscated Files or Information']

In [ ]:
word = 'citat'


mitr_df.explode('labels').query('labels=="impact" and sentence.str.contains(@word)')

# Links
- [Потом почитать про аннотирование данных](https://github.com/center-for-threat-informed-defense/tram/wiki/Data-Annotation)

# Старое

In [ ]:
add_df_l = []
prbl_l = []
for row in mitre_df.itertuples():
    part_df = pd.DataFrame({'id':row.id, 'labels':row.labels, 'url':row.url, 'proc_flag':True}, index=[0])

    sentences = [it['description'] for it in rel_l if (it['target_ref']==row.id) and (it['relationship_type']=='uses')]
    if sentences:
      add_df_l.append(pd.concat([part_df.assign(sentence=sentence) for sentence in sentences], ignore_index=True))
    else:
      prbl_l.append(part_df)



In [ ]:
mitre_df = pd.concat([mitre_df]+add_df_l, ignore_index=True)